In [1]:
# import functions
# OS interaction and time
import os
import sys
import cftime
import datetime
import time
import glob
import dask
import dask.bag as db
import calendar

# math and data
import numpy as np
import netCDF4 as nc
import xarray as xr
import scipy as sp
import scipy.linalg
from scipy.signal import detrend
import pandas as pd
import pickle as pickle
from sklearn import linear_model
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing
import statsmodels.stats.multitest as multitest

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg
from matplotlib.colors import TwoSlopeNorm

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

# random
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

In [2]:
my_era5_path = '/glade/u/home/zcleveland/scratch/ERA5/'  # path to subset data
misc_data_path = '/glade/u/home/zcleveland/scratch/misc_data/'  # path to misc data
plot_out_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/plots/'  # path to generated plots
scripts_main_path = '/glade/u/home/zcleveland/NAM_soil-moisture/scripts_main/'  # path to my dicts, lists, and functions

In [3]:
# import variable lists and dictionaries
if scripts_main_path not in sys.path:
    sys.path.insert(0, scripts_main_path)  # path to file containing these lists/dicts
import my_dictionaries

# my lists
sfc_instan_list = my_dictionaries.sfc_instan_list  # instantaneous surface variables
sfc_accumu_list = my_dictionaries.sfc_accumu_list  # accumulated surface variables
pl_var_list = my_dictionaries.pl_var_list  # pressure level variables
invar_var_list = my_dictionaries.invar_var_list  # invariant variables
NAM_var_list = my_dictionaries.NAM_var_list  # NAM-based variables
region_avg_list = my_dictionaries.region_avg_list  # region IDs for regional averages
flux_var_list = my_dictionaries.flux_var_list  # flux variables that need to be flipped (e.g., sensible heat so that it's positive up instead of down
misc_var_list = my_dictionaries.misc_var_list  # misc variables

# my dictionaries
var_dict = my_dictionaries.var_dict  # variables and their names
var_units = my_dictionaries.var_units  # variable units
region_avg_dict = my_dictionaries.region_avg_dict  # region IDs and names
region_avg_coords = my_dictionaries.region_avg_coords  # coordinates for regions
region_colors_dict = my_dictionaries.region_colors_dict  # colors to plot for each region

# import common functions that I've created
from get_var_data import get_var_data, get_var_files, open_var_data, subset_var_data
from my_functions import month_num_to_name, ensure_var_list

In [4]:
# define a funciton to plot a time series of ERA5 vs ESA soil moisture over the Colorado Plateau
def plot_timeseries_sm_cp(overwrite_flag=False):
    # create out filename and path -- check existence
    out_fn = 'ERA5_ESA_sm_cp_ts.png'
    out_fp = os.path.join(plot_out_path, 'time_series', out_fn)

    if overwrite_flag:
        pass
    elif os.path.exists(out_fp):
        print(f'{out_fn} already exists.  Set overwrite_flag=True to overwrite.')
        print('Skipping . . .\n')
        return
    else:
        print(f'Starting plot for {out_fn}')

    # get file path for datasets
    era_file = f'{scratch_path}ERA5/cp/swvl1_198001_201912_cp.nc'
    regrid_file = f'{scratch_path}ERA5/cp/swvl1_regrid_198001_2019_cp.nc'
    esa_file = f'{scratch_path}ESA_data/cp/sm_esa_198001_201912_cp.nc'

    # open datasets
    era_ds = xr.open_dataset(era_file)
    regrid_ds = xr.open_dataset(regrid_file)
    esa_ds = xr.open_dataset(esa_file)

    # set up figure
    plt.figure(figsize=(12,6))
    era_ds['SWVL1_AVG_CP'].plot(label='Original ERA5 Data', color='r')
    # regrid_ds['SWVL1_AVG_CP'].plot(label='Regrid ERA5 data', color='b')
    esa_ds['sm_CP'].plot(label='ESA Data', color='g')
    plt.title('ERA5 vs. ESA\nSoil Moisture')
    plt.legend()
    plt.xlabel('Time (months)')
    plt.ylabel('Soil Moisture Content (m$^3$ m$^{-3}$)')
    plt.tight_layout()
    plt.savefig(out_fp, dpi=600, bbox_inches='tight')
    plt.show()
    plt.close()

In [5]:
# plot_timeseries_sm_cp(overwrite_flag=False)

In [6]:
# define a function to plot a time series of given variables
def main_time_series(var_list, region, **kwargs):

    # get kwargs
    start_date = kwargs.get('start_date', 198001)
    end_date = kwargs.get('end_date', 201912)
    var_month_list = kwargs.get('var_month_list', [i for i in range(1,13)])
    level = kwargs.get('level', 500)
    color_cycle = kwargs.get('color_cycle', plt.rcParams['axes.prop_cycle'].by_key()['color'])

    # split start and end date to get year and month
    start_year, start_month = f'{start_date}'[:4], f'{start_date}'[4:]
    end_year, end_month = f'{end_date}'[:4], f'{end_date}'[4:]

    var_month_list = ensure_var_list(var_month_list)
    var_list = ensure_var_list(var_list)

    # months list
    var_months = month_num_to_name(var=var_list[0], months=var_month_list, **kwargs)

    var_data_list = []

    for var in var_list:
        # get var files
        var_files = get_var_files(var, region, **kwargs)
        var_da = open_var_data(var_files, var, type='da', **kwargs)

        # open dataset
        var_data = subset_var_data(var_files, var, var_month_list, region, da=var_da, mean_flag=True, **kwargs)
        var_data_list.append(var_data)

    # plot time series
    fig, ax1 = plt.subplots(figsize=(12,6), facecolor='white')

    color = color_cycle[0]
    ax1.plot(var_data_list[0], label=var_dict[var_list[0]], color=color)
    # customize plot
    ax1.set_xlabel('Year')
    years = np.arange(1980,2020)
    ax1.set_xticks((years-1980)[::4])
    ax1.set_xticklabels([str(i) for i in years][::4])
    ax1.tick_params(axis='x', labelrotation=45)
    ax1.set_ylabel(var_dict[var_list[0]], color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.grid(True)

    if len(var_data_list) != 1:
        axes = [ax1]
        for i in range(1, len(var_list)):
            ax = ax1.twinx()
            ax.spines['right'].set_position(('outward', 60*(i-1)))
            color=color_cycle[i]
            ax.plot(var_data_list[i], label=var_dict[var_list[i]], color=color)
            ax.set_ylabel(var_dict[var_list[i]], color=color)
            ax.tick_params(axis='y', labelcolor=color)

            # ax.set_facecolor('white')

            axes.append(ax)
            ax.spines['right'].set_visible(True)

        for ax in axes:
            ax.legend(loc='lower right')

    fig.tight_layout()

    # show plot
    plt.show()


In [7]:
# define a function to plot a time series of a given variable
def plot_time_series(var_data, **kwargs):
    i = kwargs.get('i', '')
    ax = ax1.twinx()
    # ax.spines['right'].set_position(('outward', 60))
    ax.plot(var_data, label=var_dict[var_list[i]])
    ax.set_ylabel(var_dict[var_list[i]])

    axes.append(ax)